<a href="https://colab.research.google.com/github/quoctrung2005/TTNT/blob/main/s%E1%BA%AFp%20l%E1%BB%8Bch%20d%E1%BA%A1y%20m%C3%B4n%20To%C3%A1n%20v%C3%A0%20Tin%20h%E1%BB%8Dc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import random

teachers = {
    "T1": "Toán",
    "T2": "Toán",
    "T3": "Tin",
    "T4": "Tin",
}

subject_slots = {
    "Toán": 12,
    "Tin": 10
}

TOTAL_SLOTS = 25


# --------------------------
# TẠO CÁ THỂ NGẪU NHIÊN
# --------------------------
def random_individual():
    individual = [None] * TOTAL_SLOTS
    slots = list(range(TOTAL_SLOTS))
    random.shuffle(slots)

    idx = 0
    for subject, count in subject_slots.items():
        for _ in range(count):
            pos = slots[idx]
            teacher = random.choice([t for t in teachers if teachers[t] == subject])
            individual[pos] = (teacher, subject)
            idx += 1

    return individual


# --------------------------
# REPAIR – SỬA SỐ TIẾT
# --------------------------
def repair(individual):
    # Đếm số tiết hiện tại
    current = {"Toán": 0, "Tin": 0}
    for slot in individual:
        if slot is not None:
            current[slot[1]] += 1

    # Slots trống
    empty = [i for i, s in enumerate(individual) if s is None]

    # Nếu thiếu tiết → thêm vào slot trống
    for subject, required in subject_slots.items():
        while current[subject] < required and empty:
            pos = empty.pop()
            teacher = random.choice([t for t in teachers if teachers[t] == subject])
            individual[pos] = (teacher, subject)
            current[subject] += 1

    # Nếu thừa tiết → xóa bớt
    for subject, required in subject_slots.items():
        if current[subject] > required:
            over = current[subject] - required
            for i in range(TOTAL_SLOTS):
                if over == 0:
                    break
                if individual[i] is not None and individual[i][1] == subject:
                    individual[i] = None
                    over -= 1

    return individual


# --------------------------
# FITNESS
# --------------------------
def fitness(individual):
    score = 1000

    # phạt giáo viên dạy sai môn
    for slot in individual:
        if slot is not None:
            t, s = slot
            if teachers[t] != s:
                score -= 50

    # phạt GV dạy liên tiếp 4 tiết
    for i in range(TOTAL_SLOTS - 3):
        block = individual[i:i+4]
        if all(block) and len(set(t for t, s in block)) == 1:
            score -= 20

    # phạt ngày quá tải
    for d in range(5):
        day = individual[d*5:(d+1)*5]
        if sum(s is not None for s in day) > 4:
            score -= 10

    return score


# --------------------------
# CHỌN LỌC
# --------------------------
def select(population):
    k = 3
    selected = random.sample(population, k)
    return max(selected, key=fitness)


# --------------------------
# CROSSOVER
# --------------------------
def crossover(p1, p2):
    cut = random.randint(1, TOTAL_SLOTS - 2)
    child = p1[:cut] + p2[cut:]
    return repair(child)


# --------------------------
# MUTATION
# --------------------------
def mutate(individual, rate=0.1):
    for i in range(TOTAL_SLOTS):
        if random.random() < rate:
            # mutate nhưng giữ môn hợp lệ
            subject = random.choice(list(subject_slots.keys()))
            teacher = random.choice([t for t in teachers if teachers[t] == subject])
            individual[i] = (teacher, subject)
    return repair(individual)


# --------------------------
# GENETIC ALGORITHM
# --------------------------
def genetic_algorithm(generations=200, population_size=50):
    population = [random_individual() for _ in range(population_size)]

    for gen in range(generations):
        new_population = []

        # elitism
        best = max(population, key=fitness)
        new_population.append(best)

        # tạo thế hệ mới
        while len(new_population) < population_size:
            p1 = select(population)
            p2 = select(population)
            child = crossover(p1, p2)
            child = mutate(child)
            new_population.append(child)

        population = new_population

        if gen % 20 == 0:
            print(f"Gen {gen}, best fitness = {fitness(best)}")

    return max(population, key=fitness)


# --------------------------
# CHẠY THUẬT TOÁN
# --------------------------
best = genetic_algorithm()

days = ["Thứ 2", "Thứ 3", "Thứ 4", "Thứ 5", "Thứ 6"]

print("\n===== LỊCH DẠY TỐI ƯU =====")
for d in range(5):
    print(f"\n{days[d]}:")
    for p in range(5):
        slot = best[d*5 + p]
        if slot is None:
            print(f"  Tiết {p+1}: trống")
        else:
            print(f"  Tiết {p+1}: {slot[1]} — GV {slot[0]}")

print("\nFitness =", fitness(best))


Gen 0, best fitness = 980
Gen 20, best fitness = 980
Gen 40, best fitness = 980
Gen 60, best fitness = 980
Gen 80, best fitness = 980
Gen 100, best fitness = 980
Gen 120, best fitness = 980
Gen 140, best fitness = 980
Gen 160, best fitness = 980
Gen 180, best fitness = 980

===== LỊCH DẠY TỐI ƯU =====

Thứ 2:
  Tiết 1: Tin — GV T4
  Tiết 2: Toán — GV T2
  Tiết 3: Toán — GV T2
  Tiết 4: Toán — GV T1
  Tiết 5: Tin — GV T4

Thứ 3:
  Tiết 1: Toán — GV T2
  Tiết 2: Tin — GV T4
  Tiết 3: Toán — GV T2
  Tiết 4: Toán — GV T1
  Tiết 5: Toán — GV T1

Thứ 4:
  Tiết 1: Tin — GV T3
  Tiết 2: Tin — GV T3
  Tiết 3: Tin — GV T4
  Tiết 4: Tin — GV T3
  Tiết 5: trống

Thứ 5:
  Tiết 1: trống
  Tiết 2: Toán — GV T1
  Tiết 3: Tin — GV T4
  Tiết 4: Toán — GV T1
  Tiết 5: Toán — GV T1

Thứ 6:
  Tiết 1: Toán — GV T1
  Tiết 2: Tin — GV T4
  Tiết 3: trống
  Tiết 4: Tin — GV T3
  Tiết 5: Toán — GV T1

Fitness = 980
